In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
import pickle
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels as sm



In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):

	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
        names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


In [3]:
data = pd.read_csv('../Bayesian_RNN/sample_data.csv')
data.drop('index', axis=1)

data = series_to_supervised(data, n_in = 1, n_out=1, dropnan=True)

In [5]:
display(data.head())

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var42(t),var43(t),var44(t),var45(t),var46(t),var47(t),var48(t),var49(t),var50(t),var51(t)
1,0.0,0.0,0.0,1.0,103.774345,82.492828,7.0,0.0,c2a18a0d6112cc34,-512.259766,...,0,0,18,0,0,0,0,0,0,103.774345
2,1.0,0.0,0.0,1.0,103.774345,82.492828,7.0,0.0,c2a18a0d6112cc34,-512.259766,...,0,0,18,0,0,0,0,0,0,103.774345
3,2.0,0.0,0.0,1.0,103.774345,82.492828,7.0,0.0,c2a18a0d6112cc34,-512.259766,...,0,0,18,0,0,0,0,0,0,103.774345
4,3.0,0.0,0.0,1.0,103.774345,82.492828,7.0,0.0,c2a18a0d6112cc34,-512.259766,...,0,0,18,0,0,0,0,0,0,103.774345
5,4.0,0.0,0.0,1.0,103.774345,82.492828,7.0,0.0,c2a18a0d6112cc34,-512.259766,...,0,0,18,0,0,0,0,0,0,103.774345
